<a href="https://colab.research.google.com/github/ahmabboud/Pizza-Coocking-Level-Recognition/blob/main/Pizza_Doneness_Logic_Real_Time_Continuous_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install hmmlearn
#!pip install --upgrade opencv-python
!apt-get -qq install -y libsm6 libxext6 && pip install -q -U opencv-python

     |████████████████████████████████| 368kB 4.2MB/s 
     |████████████████████████████████| 49.4MB 81kB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.


In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
import fastai
from fastai import *
from fastai.vision import *
import matplotlib.pyplot as plt
from torchvision import transforms

import cv2
import os

import pandas as pd
import numpy as np

import json
from datetime import datetime
import time

from hmmlearn import hmm

import warnings
warnings.filterwarnings("ignore")

#Define required paths for images and model.

In [ ]:
root_path = Path('/content/gdrive/My Drive/Pizza/')
project_path = Path(root_path/'oven-camera-validation/')
image_path = Path(root_path/'oven-camera-validation/')
model_path = Path(root_path/'models/')
image_path_sample = Path(image_path/'D828C929CA75/c1e587aa-580e-4485-905c-4ca611b42493/')
#image_path_sample = Path(image_path/'D828C929E2E9/75b4dc01-7e66-47eb-95dd-6b4435ca9df3/')
#image_path_sample = Path(image_path/'D828C929E2F9/962ffbd5-cb2e-4b0c-8e69-a22262250b24/')
#image_path_sample = Path(image_path/'D828C936B06D/0be853f9-a169-461d-b9c0-0e032bc78aa5/')
#image_path_sample = Path(image_path/'D828C933D251/ef5e733c-7d7c-42b4-9901-ffac4dcdbfdb/')
#image_path_sample = Path(image_path/'D828C929E2FC/3ed6c52f-cc12-43ab-884f-82d75c8eb39a/')
#image_path_sample = Path(image_path/'D828C933D251/ef5e733c-7d7c-42b4-9901-ffac4dcdbfdb/')


##Here we read image files from the image folder along with their corespondeing meta data, sort them based on their time stamp, and store their filename along with the corresponding time stamp in a dictionary called 'sorted_images_dict'. 

* In practice, we will have a real time camera feed instead of a folder of images.

In [ ]:
result_dict = {}
for file_name in os.listdir(image_path_sample):
    if file_name.endswith('.json'):
        full_path_to_file = os.path.join(image_path_sample,file_name)
        with open(full_path_to_file) as f:
            meta_data_dict = json.load(f)
            cooking_date = int(meta_data_dict['Metadata']['epochtimestamp'])
            datetime_object =  datetime.fromtimestamp(cooking_date/1000)
            #datetime_object =  datetime(*time.gmtime(cooking_date/1000.)[:6])
            #print(time.strftime('%m/%d/%Y %H:%M:%S', time.gmtime(cooking_date/1000.)), datetime_object)
            result_dict[file_name] = datetime_object

sorted_images_dict = sorted(result_dict.items(), key=lambda x: x[1])
print(sorted_images_dict)

[('8837b2f2-6f16-4e42-889c-5f89147ba47d.json', datetime.datetime(2020, 8, 10, 22, 40, 31, 232000)), ('e80c81d9-8c6b-4118-8ef7-ebad990e5548.json', datetime.datetime(2020, 8, 10, 22, 40, 41, 298000)), ('6fed65f6-e96a-45ff-88a7-4102af79f98c.json', datetime.datetime(2020, 8, 10, 22, 40, 51, 355000)), ('0d552bec-c6e2-4541-9410-8d7aea7c7187.json', datetime.datetime(2020, 8, 10, 22, 41, 1, 364000)), ('ce1ce25e-91f9-491e-b624-873db0e5c97d.json', datetime.datetime(2020, 8, 10, 22, 41, 11, 374000)), ('394d67e3-2fae-4d9d-8573-44d60753b0cf.json', datetime.datetime(2020, 8, 10, 22, 41, 21, 374000)), ('9851ad7c-3708-409d-aec0-b6c69a945da3.json', datetime.datetime(2020, 8, 10, 22, 41, 31, 379000)), ('9a70db2c-4ad6-4b06-82f1-5524f07a159b.json', datetime.datetime(2020, 8, 10, 22, 41, 41, 433000)), ('7f4da2ad-a097-4a76-a898-382548fa5eff.json', datetime.datetime(2020, 8, 10, 22, 41, 51, 463000)), ('6805662f-7596-4f7a-9e9d-f1dfb9aaf37b.json', datetime.datetime(2020, 8, 10, 22, 42, 1, 525000)), ('83a5c4d4-

##A generator function is designed here to provide the camera feed images to our computer vision models and oven operation logic module.

In [ ]:
def get_image_generator(sorted_images_dict, image_folder):
  def get_next_image():
    for img_json, timestamp in sorted_images_dict:
      #print('elapsed time: ', elapsed_time, 'timestamp: ', str(timestamp), 'first time: ', str(first_frame_time))
      #print(f'timestamp: {str(timestamp)}, image json : {str(img_json)}')
      img_name = img_json.replace('.json', '.jpg')

      # read in image as cv2 numpy array
      frame = cv2.imread(os.path.join(image_folder, img_name))

      # convert frame from cv2 numpy array to PIL tensor
      img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
      tensor = pil2tensor(img, dtype=np.uint8)
      tensor = tensor.float() / 255.

      # store as fastai image
      fastai_frame = fastai.vision.image.Image(tensor)
      yield (img, fastai_frame, timestamp, image_folder, img_name)
  
  return get_next_image

#This function provides our model predictions for various computer vision models such as 'pizza/non-pizza' or 'pizza doneness' model.

In [ ]:
def get_predictor(model_path, pizza_model_name):
  predicted_categories, predicted_probabilities, frame_path_list, frame_name_list, timestamp_list = [], [], [], [], []
  def get_image_prediction(fastai_frame, image_folder, img_name, timestamp):
    learner = load_learner(model_path, pizza_model_name)

    category, _, probs = learner.predict(fastai_frame)
    return (str(category), list(probs), os.path.join(image_folder, img_name), img_name, timestamp)
   
  return get_image_prediction

#We then create an incremental data frame from the camera feed. This data frame is then used to adjust our models predictions considering the uncertainty of these model outputs.

In [ ]:
def get_prediction_dataframe(image_folder, sorted_images_dict, model_path, pizza_model_name, category_col_name, category_col_categories):
  predicted_categories, predicted_probabilities, frame_path_list, frame_name_list, timestamp_list = [], [], [], [], []

  get_image_prediction = get_predictor(model_path, pizza_model_name)
  get_next_image = get_image_generator(sorted_images_dict, image_path_sample)
  for img, fastai_frame, timestamp, image_folder, img_name in get_next_image():
    category_str, probs_list, image_path, img_name, timestamp = get_image_prediction(fastai_frame, image_folder, img_name, timestamp)
    predicted_categories, predicted_probabilities, frame_path_list, frame_name_list, timestamp_list = map(operator.add,
                          [predicted_categories, predicted_probabilities, frame_path_list, frame_name_list, timestamp_list], 
                          [[category_str], [probs_list], [image_path], [img_name], [timestamp]])
    df = pd.DataFrame({'timestamp': timestamp_list, 'frame_path':frame_path_list, 'frame_name':frame_name_list, 'category':predicted_categories, 'probs':predicted_probabilities})
    df1 = pd.DataFrame(df['probs'].tolist(), index=df.index)
    df1.columns=[category_col_name + '_Prob_' + str(i) for i in range(len(df1.columns))]
    df[category_col_name] = df['category'].astype('category').cat.set_categories(category_col_categories)
    yield pd.concat([df, df1], axis=1)

##Different strategies for adjustement of computer vision model predictions. Two strategies are implemented so far: 1) HMM based on discrete predictions and 2) Moving average on prediction probabilities of different categories.

##In future, Klamn filter and Particle filters might produce more accurate predictions.

In [ ]:
def normalize(x):
    return x / x.sum()

def adjust_continuous(prediction_data, column_name, column_categories):
  prob_columns = [column for column in prediction_data.columns if column_name+'_Prob_' in column]
  #prediction_data = prediction_data.assign(**{prob+'_adjusted':prediction_data[prob].rolling(3).mean() for prob in prob_columns})
  prediction_data = prediction_data.assign(**{prob+'_adjusted':prediction_data[prob].rolling(3).apply(np.mean) for prob in prob_columns})
  prob_columns_adjusted = [prob+'_adjusted' for prob in prob_columns]
  prediction_data[column_name + '_adjusted_values'] = column_categories[np.argmax(prediction_data[prob_columns_adjusted].tail(1))]
  return prediction_data

def adjust_with_HMM(prediction_data, column_name, column_categories, start_probs, trans_matrix, emission_probs, num_states):
  model = hmm.MultinomialHMM(n_components=num_states)
  model.startprob_ = np.array(start_probs)
  model.transmat_ = np.array(trans_matrix)
  model.emissionprob_ = np.array(emission_probs)

  X = np.atleast_2d(prediction_data[column_name].cat.codes).T
  corrected_predictions = list(model.decode(X)[1])
  prediction_data[column_name + '_adjusted'] = corrected_predictions
  prediction_data[column_name + '_adjusted_values'] = prediction_data[column_name].cat.categories[prediction_data[column_name+'_adjusted']]

def create_predictor_string(latest_predictions, category_col_name_list):
  if latest_predictions['empty_or_full'][1].lower() == 'empty_oven':
    return 'Empty oven'
  if latest_predictions['pizza_nonpizza'][1].lower() == 'non-pizza':
    return 'Not a pizza'
  if latest_predictions['raw_or_frozen'][1].lower() == 'scratch':
    return 'Raw pizza'
  return latest_predictions['pizza_doneness'][1]


#We can create a list of models along with their corresponding HMM parameters to adjust their predictions. With this code structure, we can have arbitrary models contributing to the oven operation logic (currently summarized in 'create_predictor_string' funtion).

In [ ]:
model_list = ['platform_oven_empty_or_full.pkl', 'platform_pizza_classification.pkl']
category_col_name_list = ['empty_or_full', 'pizza_doneness']
categories_col_name_list = [('empty_oven', 'food_oven'), ('raw', 'light', 'medium', 'dark')]

category_col_name_HMM_paramlist = {'empty_or_full': {'start_probs': [0.5, 0.5], 
                                                 'trans_matrix': [[0.9, 0.1], [0.1, 0.9]], 
                                                 'emission_probs': [[0.9, 0.1], [0.1, 0.9]],
                                                 'num_states': 2
                                                 }, 
                               'pizza_nonpizza': {'start_probs': [0.5, 0.5], 
                                                 'trans_matrix': [[0.9, 0.1], [0.1, 0.9]], 
                                                 'emission_probs': [[0.9, 0.1], [0.1, 0.9]],
                                                 'num_states': 2
                                                 }, 
                               'raw_or_frozen': {'start_probs': [0.5, 0.5], 
                                                 'trans_matrix': [[0.9, 0.1], [0.1, 0.9]], 
                                                 'emission_probs': [[0.9, 0.1], [0.1, 0.9]],
                                                 'num_states': 2
                                                 }, 
                               'pizza_doneness': {'start_probs': [0.7, 0.1, 0.1, 0.1], 
                                                 'trans_matrix': [[0.7, 0.1, 0.1, 0.1], [0.1, 0.7, 0.1, 0.1], [0.1, 0.1, 0.7, 0.1], [0.1, 0.1, 0.1, 0.7]], 
                                                 'emission_probs': [[0.7, 0.1, 0.1, 0.1], [0.1, 0.7, 0.1, 0.1], [0.1, 0.1, 0.7, 0.1], [0.1, 0.1, 0.1, 0.7]],
                                                 'num_states': 4
                                                 }
                               }

x = map(get_prediction_dataframe, [image_path_sample] * len(model_list), [sorted_images_dict] * len(model_list), 
        [model_path] * len(model_list), model_list, category_col_name_list, categories_col_name_list)
z=list(x)

prediction_list = np.empty((len(model_list), 0)).tolist()

not_empty = True
index = 0
while not_empty: #and index<30:
  df_list = []
  latest_predictions = {}
  index += 1
  for i, y in enumerate(z):
    try:
      df = next(y)
      if index > 3:
        adjust_with_HMM(df, category_col_name_list[i], categories_col_name_list[i], **category_col_name_HMM_paramlist[category_col_name_list[i]])
        #df = adjust_continuous(df, category_col_name_list[i], categories_col_name_list[i])
        latest_prediction = list(df.tail(1)[[category_col_name_list[i], category_col_name_list[i]+'_adjusted_values']].values)
        prediction_list[i].append(latest_prediction)
        latest_predictions[category_col_name_list[i]] = latest_prediction[0]
        df_list.append(df)
    except StopIteration:
      not_empty=False
  if latest_predictions.keys():
    print(f'======={create_predictor_string(latest_predictions, category_col_name_list)}========')


=======light========
=======light========
=======light========
=======light========
=======light========
=======light========
=======light========
=======light========
=======light========
=======light========
=======light========
=======light========
=======light========
=======light========
=======light========
=======light========
=======light========
=======light========
=======light========
=======light========
=======light========
=======light========
=======light========
=======light========
=======light========
=======light========
=======light========
=======light========
=======light========
=======light========
=======light========
=======light========
=======light========
=======light========
=======light========
=======light========
=======light========
=======light========
=======light========
=======light========
=======light========
=======light========
=======light========
=======light========
=======light========
=======light========
=======light========
=======light=